In [1]:
import tensorflow as tf

In [2]:
graph = tf.Graph()

In [3]:
with graph.as_default():
    with tf.name_scope("Variabels"):
        global_step = tf.Variable(0,dtype=tf.int32,trainable=False,name="global_step")
        total_output = tf.Variable(0.0,dtype=tf.float32,trainable=False,name="total_output")
        
    with tf.name_scope("Transformation"):
        with tf.name_scope("input"):
            a = tf.placeholder(tf.float32,shape=[None],name="input_placeholder_a")

        with tf.name_scope("intermediate_layer"):
            b = tf.reduce_prod(a,name="prod")
            c = tf.reduce_sum(a,name="sum")
                
        with tf.name_scope("output"):
            output = tf.add(b,c,name="output")
            
    with tf.name_scope("update"):
            update_total = total_output.assign_add(output)
            increment_step = global_step.assign_add(1)
                
    with tf.name_scope("summaries"):
            avg = tf.div(update_total,tf.cast(increment_step,tf.float32),name="average")
            
            tf.summary.scalar("output_summary",output,b'Output')
            tf.summary.scalar("total_summary",update_total,b'Sum of outputs over time')
            tf.summary.scalar("average_summary",avg,b'Average of ouputs over time')
        
    with tf.name_scope("global_ops"):
            init = tf.global_variables_initializer()
            merged_summaries = tf.summary.merge_all()

In [4]:
sess = tf.Session(graph=graph)
writer = tf.summary.FileWriter('./improved_graph',graph)
sess.run(init)

In [5]:
def run_graph(input_tensor):
    input_dict={a:input_tensor}
    _,step,summary = sess.run([output,increment_step,merged_summaries], feed_dict=input_dict)
    writer.add_summary(summary,gobal_step=step)

In [6]:
run_graph([3,5])
run_graph([3,1,3,3])
run_graph([8])

TypeError: Fetch argument None has invalid type <class 'NoneType'>

In [7]:
writer.flush()
writer.close()
sess.close()